In [1]:
!pip install ultralytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 592.7/592.7 kB 12.6 MB/s eta 0:00:00


In [2]:
import os
import shutil
import random
import cv2
!pip install tqdm --upgrade
from tqdm.notebook import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
train_path_img = "./yolo_data/images/train/"
train_path_label = "./yolo_data/labels/train/"
val_path_img = "./yolo_data/images/val/"
val_path_label = "./yolo_data/labels/val/"
test_path = "./yolo_data/test"

In [5]:
def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images
    

    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)

    
    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')
        
    

    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")
    
    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')
            
        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")
    
        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")
    


    ### copytin images to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")
    
    print("------ TASK COMPLETED -------")

## spliting the data into train-test and creating train.txt and test.txt files
# train_test_split('/content/drive/MyDrive/custom_notebooks/yolo_data/')

### for label_tag
train_test_split('/content/drive/MyDrive/yolov8/dataset/') ### without negative images

------ PROCESS STARTED -------
--- This folder has a total number of 70 images---


  0%|          | 0/56 [00:00<?, ?it/s]

------ Training data created with 80% split 56 images -------


  0%|          | 0/14 [00:00<?, ?it/s]

------ Testing data created with a total of 14 images ----------
------ TASK COMPLETED -------


In [6]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.110 🚀 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 23.3/78.2 GB disk)


In [7]:
!yolo task=detect mode=train model=yolov8x.pt data=/content/drive/MyDrive/yolov8/dataset.yaml epochs=10 imgsz=640 batch=8 project=/content/drive/MyDrive/yolov8/training_results name=shoplifting 

100% 131M/131M [00:00<00:00, 240MB/s]
Ultralytics YOLOv8.0.110 🚀 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/content/drive/MyDrive/yolov8/dataset.yaml, epochs=10, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/yolov8/training_results, name=shoplifting, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None

In [8]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/yolov8/training_results/shoplifting/weights/best.pt conf=0.55 source=/content/drive/MyDrive/yolov8/test_images

Ultralytics YOLOv8.0.110 🚀 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 268 layers, 68124531 parameters, 0 gradients

image 1/6 /content/drive/MyDrive/yolov8/test_images/s1.jpg: 448x640 1 shoplifting, 75.1ms
image 2/6 /content/drive/MyDrive/yolov8/test_images/s13.jpg: 448x640 1 shoplifting, 70.7ms
image 3/6 /content/drive/MyDrive/yolov8/test_images/s14.jpg: 448x640 1 shoplifting, 70.7ms
image 4/6 /content/drive/MyDrive/yolov8/test_images/s15.jpg: 448x640 1 shoplifting, 70.7ms
image 5/6 /content/drive/MyDrive/yolov8/test_images/s16.jpg: 448x640 1 shoplifting, 70.7ms
image 6/6 /content/drive/MyDrive/yolov8/test_images/s3.jpg: 448x640 2 shopliftings, 70.7ms
Speed: 2.1ms preprocess, 71.5ms inference, 40.1ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict


In [9]:
!cp -r /content/runs/detect/predict /content/drive/MyDrive/yolov8/output

In [10]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/yolov8/training_results/shoplifting/weights/best.pt conf=0.55 source=/content/drive/MyDrive/yolov8/test_videos

Ultralytics YOLOv8.0.110 🚀 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 268 layers, 68124531 parameters, 0 gradients

video 1/1 (1/4029) /content/drive/MyDrive/yolov8/test_videos/VIDEO Brazen Shoplifter At SF Walgreens Caught On Video.mp4: 384x640 (no detections), 67.7ms
video 1/1 (2/4029) /content/drive/MyDrive/yolov8/test_videos/VIDEO Brazen Shoplifter At SF Walgreens Caught On Video.mp4: 384x640 (no detections), 39.7ms
video 1/1 (3/4029) /content/drive/MyDrive/yolov8/test_videos/VIDEO Brazen Shoplifter At SF Walgreens Caught On Video.mp4: 384x640 (no detections), 39.4ms
video 1/1 (4/4029) /content/drive/MyDrive/yolov8/test_videos/VIDEO Brazen Shoplifter At SF Walgreens Caught On Video.mp4: 384x640 (no detections), 39.3ms
video 1/1 (5/4029) /content/drive/MyDrive/yolov8/test_videos/VIDEO Brazen Shoplifter At SF Walgreens Caught On Video.mp4: 384x640 (no detections), 34.3ms
video 1/1 (6/4029) /content/drive/MyDrive/yolov8/test_videos/VIDEO Brazen

In [11]:
!cp -r /content/runs/detect/predict2 /content/drive/MyDrive/yolov8/output

In [ ]:
! pip install imageAI
! pip install opencv-python
! pip install Pillow
! pip install numpy

from imageai.Detection import ObjectDetection

obj_detect = ObjectDetection()



['/bin/bash: -c: line 0: syntax error near unexpected token `source="0",\'', '/bin/bash: -c: line 0: `model.predict(source="0", show=True)\'']
